<a href="https://colab.research.google.com/github/Bryan-Az/RL-SARSA-Gym/blob/main/reinforcement_sarsa_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning using SARSA in OpenAI Gym / Gymnasium

## Imports and Installs

In [1]:
!pip install gymnasium[atari] # for the gym library
!pip install gymnasium[accept-rom-license] #to add atari envs
!pip install ale-py # for atari envs
import gymnasium as gym
import numpy as np
import random
import time
from IPython.display import clear_output

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=6d2ddc978113487ce7435794daf145332e18b9afa6f0f831eeaecb92c029ab22
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


## Step 1: Creating the Environment

In [2]:
env_name = "MountainCar-v0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
env = gym.make(env_name, render_mode="rgb_array")

## Step 2: Creating the Agent with SARSA Learning

These are the hyperparameters specific to the training within the environment and learning task. These are used within the Agent / Learner class and are similar to the training of a neural network.

In [132]:
#MAX_NUM_EPISODES = 1000
MAX_NUM_EPISODES = 25000 #100001
STEPS_PER_EPISODE = 200 #  This is specific to MountainCar. May change with env
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.1  # Learning rate
GAMMA = 0.9 #1.0  # Discount factor
NUM_DISCRETE_BINS = 30  # Number of bins to Discretize each observation dim

In [144]:
class Q_Learner(object):
    def __init__(self, env):
        self.obs_shape = env.observation_space.shape
        self.obs_high = env.observation_space.high
        self.obs_low = env.observation_space.low
        # printing all the obs_ for debugging
        #print(self.obs_high, 'obs high')
        #print(self.obs_low, ' obs low')
        self.obs_bins = NUM_DISCRETE_BINS  # Number of bins to Discretize each observation dim
        self.bin_width = (self.obs_high - self.obs_low) / self.obs_bins
        self.action_shape = env.action_space.n
        # Create a multi-dimensional array (aka. Table) to represent the
        # Q-values
        self.Q = np.zeros((self.obs_bins + 1, self.obs_bins + 1,
                           self.action_shape))  # (51 x 51 x 3)
        self.alpha = ALPHA  # Learning rate
        self.gamma = GAMMA  # Discount factor
        self.epsilon = 0.9
        ### Unique to SARSA Method ###
        rewards = []
    ''' def discretize(self, obs):
        # Convert the observation to a tuple of integers using list comprehension
        # and unpack it using the * operator when indexing into self.Q
        return tuple([int(np.round((obs[i] - self.obs_low[i]) / self.bin_width[i])) for i in range(len(obs))])


    def get_action(self, obs):
        discretized_obs = self.discretize(obs)
        # Epsilon-Greedy action selection
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[discretized_obs])  # unpack the tuple
        else:  # Choose a random action
            return np.random.choice([a for a in range(self.action_shape)])
    '''
    def discretize(self, obs):
      #clipped_obs = np.clip(obs[0], self.obs_low, self.obs_high)
      #return tuple(((clipped_obs - self.obs_low) / self.bin_width).astype(int))
      #discretized_env = (self.obs_high - self.obs_low) / self.obs_bins
      #discretized_pos = int((clipped_obs[0] - self.obs_low[0]) / discretized_env[0])
      #discretized_vel = int((clipped_obs[1] - self.obs_low[1]) / discretized_env[1])
      #return discretized_pos, discretized_vel
      discrete_obs = (obs[0] - self.obs_low) / self.bin_width
      return tuple(np.clip(discrete_obs.astype(int), 0, self.obs_bins - 1))

    def get_action(self, obs):
        discretized_obs = self.discretize(obs)
        # Epsilon-Greedy action selection
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[discretized_obs])
        else:  # Choose a random action
            return np.random.choice([a for a in range(self.action_shape)])

    def learn(self, obs, action, reward, next_obs, next_action):
        '''
        This is the SARSA learning method that uses the get_action function
        to retrieve the next state-action
        as input to updating the Q learning matrix, versus the
        max Q value of the next state as in Q Learning.
        '''
        discretized_obs = self.discretize(obs)
        discretized_next_obs = self.discretize(next_obs)
        # change self.Q[discretized_next_obs][next_action] to np.max(self.Q[discretized_next_obs])
        # if want to use Q learning
        td_target = reward + self.gamma * self.Q[discretized_next_obs][next_action]
        td_error = td_target - self.Q[discretized_obs][action]
        self.Q[discretized_obs][action] += self.alpha * td_error

def train(agent, env):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        done = False
        obs = env.reset()
        action = agent.get_action(obs)
        ### printing the obs for debugging
        #print(obs, ' obs')
        #print(type(obs), ' obs type')
        #print(obs[0][0], ' obs[0]')
        #print(obs[0][1], ' obs[1]')
        #print(obs[1], ' obs[1]')
        #print(type(obs[0]), ' obs[0] type')
        #print(type(obs[1]), ' obs[1] type')
        total_reward = 0.0
        while not done:
            next_obs, reward, terminated, truncated, info = env.step(action)
            # retrieving the next action necessary for the SARSA learning method
            next_action = agent.get_action(next_obs)
            done = terminated or truncated
            agent.learn(obs, action, reward, next_obs, next_action)
            obs = next_obs
            action = next_action
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
                                     total_reward, best_reward, agent.epsilon))
    # Return the trained policy
    return np.argmax(agent.Q, axis=2)


def test(agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    while not done:
        action = policy[agent.discretize(obs)]
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        obs = next_obs
        total_reward += reward
    return total_reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Step 3: Instantiating & Training the Learner / Agent
In this stage, the agent is still learning its' preference / policy. In the testing phase, the outcome of its' preferences will be monitored and visualized.

In [145]:
agent = Q_Learner(env)
learned_policy = train(agent, env)

Moviepy - Building video /content/gym_monitor_output/rl-video-episode-1000.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-1000.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-1000.mp4
Episode#:0 reward:-200.0 best_reward:-200.0 eps:0.8998000000000165
Episode#:1 reward:-200.0 best_reward:-200.0 eps:0.8996000000000329
Episode#:2 reward:-200.0 best_reward:-200.0 eps:0.8994000000000494
Episode#:3 reward:-200.0 best_reward:-200.0 eps:0.8992000000000658
Episode#:4 reward:-200.0 best_reward:-200.0 eps:0.8990000000000823
Episode#:5 reward:-200.0 best_reward:-200.0 eps:0.8988000000000987
Episode#:6 reward:-200.0 best_reward:-200.0 eps:0.8986000000001152
Episode#:7 reward:-200.0 best_reward:-200.0 eps:0.8984000000001316
Episode#:8 reward:-200.0 best_reward:-200.0 eps:0.8982000000001481
Episode#:9 reward:-200.0 best_reward:-200.0 eps:0.8980000000001646
Episode#:10 reward:-200.0 best_reward:-200.0 eps:0.897800000000181
Episode#:11 reward:-200.0 best_reward:-200.0 eps:0.8976000000001975
Episode#:12 reward:-200.0 best_reward:-200.0 eps:0.8974000000002139
Episode#:13 reward:-200.0 best_rew

Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-101000.mp4
Moviepy - Building video /content/gym_monitor_output/rl-video-episode-100000.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-100000.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-100000.mp4
Episode#:528 reward:-200.0 best_reward:-200.0 eps:0.7942000000087038
Episode#:529 reward:-200.0 best_reward:-200.0 eps:0.7940000000087203
Episode#:530 reward:-200.0 best_reward:-200.0 eps:0.7938000000087367
Episode#:531 reward:-200.0 best_reward:-200.0 eps:0.7936000000087532
Episode#:532 reward:-200.0 best_reward:-200.0 eps:0.7934000000087696
Episode#:533 reward:-200.0 best_reward:-200.0 eps:0.7932000000087861
Episode#:534 reward:-200.0 best_reward:-200.0 eps:0.7930000000088026
Episode#:535 reward:-200.0 best_reward:-200.0 eps:0.792800000008819
Episode#:536 reward:-200.0 best_reward:-200.0 eps:0.7926000000088355
Episode#:537 reward:-200.0 best_reward:-200.0 eps:0.7924000000088519
Episode#:538 reward:-200.0 best_reward:-200.0 eps:0.7922000000088684
Episode#:539 reward:-200.0 best_reward:-200.0 eps:0.7920000000088848
Episode#:540 reward:-200.0 best_reward:-200.0 eps:0.7918000000089013
Episode#:

Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-2000.mp4
Episode#:1000 reward:-200.0 best_reward:-200.0 eps:0.6998000000164698
Episode#:1001 reward:-200.0 best_reward:-200.0 eps:0.6996000000164863
Episode#:1002 reward:-200.0 best_reward:-200.0 eps:0.6994000000165027
Episode#:1003 reward:-200.0 best_reward:-200.0 eps:0.6992000000165192
Episode#:1004 reward:-200.0 best_reward:-200.0 eps:0.6990000000165356
Episode#:1005 reward:-200.0 best_reward:-200.0 eps:0.6988000000165521
Episode#:1006 reward:-200.0 best_reward:-200.0 eps:0.6986000000165685
Episode#:1007 reward:-200.0 best_reward:-200.0 eps:0.698400000016585
Episode#:1008 reward:-200.0 best_reward:-200.0 eps:0.6982000000166014
Episode#:1009 reward:-200.0 best_reward:-200.0 eps:0.6980000000166179
Episode#:1010 reward:-200.0 best_reward:-200.0 eps:0.6978000000166343
Episode#:1011 reward:-200.0 best_reward:-200.0 eps:0.6976000000166508
Episode#:1012 reward:-200.0 best_reward:-200.0 eps:0.697400000016667

...

Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-124000.mp4
Moviepy - Building video /content/gym_monitor_output/rl-video-episode-123000.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-123000.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-123000.mp4
Episode#:23528 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23529 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23530 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23531 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23532 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23533 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23534 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23535 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23536 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23537 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23538 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23539 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:23540 reward:-200.0 

Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-25000.mp4
Episode#:24000 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24001 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24002 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24003 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24004 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24005 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24006 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24007 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24008 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24009 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24010 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24011 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24012 reward:-200.0 b

Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-125000.mp4
Moviepy - Building video /content/gym_monitor_output/rl-video-episode-124000.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-124000.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-124000.mp4
Episode#:24528 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24529 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24530 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24531 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24532 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24533 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24534 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24535 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24536 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24537 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24538 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24539 reward:-200.0 best_reward:-200.0 eps:0.004999500028947066
Episode#:24540 reward:-200.0 

## Step 4: Evaluating the Training by Recording the Agent within the Environment

In [146]:
# Uses the Gym Monitor wrapper to evalaute the agent and record video
# only one video will be saved

# video of the final episode with the episode trigger
env = gym.wrappers.RecordVideo(
    env, "./gym_monitor_output", episode_trigger=lambda x: x == 0)

test(agent, env, learned_policy)

env.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/gym_monitor_output folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/gym_monitor_output/rl-video-episode-26000.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-26000.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-26000.mp4
Moviepy - Building video /content/gym_monitor_output/rl-video-episode-0.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-0.mp4
